In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
print(tf.__version__)
np.set_printoptions(threshold=sys.maxsize)

2.10.0


<IPython.core.display.Javascript object>

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(12, activation='softmax'))


In [ ]:
model = Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    # The second convolution
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # The third convolution
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # The fourth convolution
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # The fifth convolution
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    Flatten(),
    # 512 neuron hidden layer
    Dense(512, activation='relu'),
    
    Dense(12, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 149, 149, 32)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 341056)            0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [11]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='CategoricalCrossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        r"C:\Users\Win10\Desktop\constellation_detect\data",  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')


Found 1908 images belonging to 12 classes.


In [13]:
type(train_datagen),type(train_generator)

(keras.preprocessing.image.ImageDataGenerator,
 keras.preprocessing.image.DirectoryIterator)

In [14]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Epoch 1/15
8/8 [==============================] - 240s 31s/step - loss: 5.5296 - accuracy: 0.1709
Epoch 2/15
8/8 [==============================] - 243s 30s/step - loss: 1.9616 - accuracy: 0.4248
Epoch 3/15
8/8 [==============================] - 235s 29s/step - loss: 1.2568 - accuracy: 0.6787
Epoch 4/15
8/8 [==============================] - 215s 26s/step - loss: 0.7342 - accuracy: 0.8242
Epoch 5/15
8/8 [==============================] - 242s 31s/step - loss: 0.3963 - accuracy: 0.9061
Epoch 6/15
8/8 [==============================] - 235s 30s/step - loss: 0.3306 - accuracy: 0.9307
Epoch 7/15
8/8 [==============================] - 240s 29s/step - loss: 0.1098 - accuracy: 0.9723
Epoch 8/15
8/8 [==============================] - 242s 29s/step - loss: 0.1671 - accuracy: 0.9580
Epoch 9/15
8/8 [==============================] - 199s 24s/step - loss: 0.0514 - accuracy: 0.9891
Epoch 10/15
8/8 [==============================] - 224s 28s/step - loss: 0.0389 - accuracy: 0.9911
Epoch 11/15
8/8 [==

In [15]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

In [16]:
# Define the class labels
class_labels = ['Aquarius', 'Aries', 'Cancer', 'Capricornus', 'Gemini', 'Leo', 'Libra', 'Pisces', 'Sagittarius', 'Scorpius', 'Taurus','Virgo']

In [17]:

path = r"C:\Users\Win10\Desktop\constellation_detect\istockphoto-1186727890-612x612.jpg"
# predicting images
img = load_img(path, target_size=(300, 300))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

classes = model.predict(images, batch_size=10)
predicted_class_index = np.argmax(classes[0])

predicted_label = class_labels[predicted_class_index]

# Print the predicted class label
print(predicted_label)

1/1 [==============================] - 1s 873ms/step
Gemini


In [18]:
import pickle

In [19]:
# Save the model
model.save('constellation_class.h5')
